In [3]:
import osmium
import pandas as pd
from pandas import ExcelWriter
from shapely.wkt import loads, dumps
import numpy as np
from pprint import pprint
from collections import OrderedDict
import sys, os

%matplotlib inline
wktfab = osmium.geom.WKTFactory()

In [ ]:
class HighwaysHandler(osmium.SimpleHandler):
    """osh dosyası sadece "highways" bilgisi ihtiva etmelidir."""
    
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.nodes = []
        self.ways=[]

        # dataframe tablo başlıkları
        self.node_columns = []
        self.way_columns = []
        
    def node(self, n):
        """
        Dosya içinde "node" ile her karşılaşıldığında bu fonksiyon otomatik çalışır.
        """
        # node'ların zaman bilgisi
        ts = pd.Timestamp(n.timestamp)
        if n.visible: lat, lon, visible = (n.location.lat, n.location.lon, 1)
        else: lat, lon, visible = (None, None, 0) 
        
        # dataframe için tablo başlıkları
        self.node_columns = ["type", "id", "version", "time", "user_id",
                             "latitude", "longitude", "visible"]        
        self.nodes.append(["node", n.id, n.version, ts, n.uid,
                           lat, lon, visible])
    
    def way(self, w):
        """Dosya içinde "way" ile her karşılaşıldığında bu fonksiyon otomatik çalışır."""
        # node id'lerini alıyoruz. daha sonra ways listesindeki
        # node id'ye bağlı olarak ilgili node bilgilari alınacak.
        nodes = [node.ref for node in w.nodes]
        
        # tag bilgisini alıp dictionary formatına dönüştürüyoruz.abs
        # yol ile ilgili bazı bilgileri (yol türü, veri elde etme
        # şekli gibi) buradan alacağız.
        #tags = {t.k: t.v for t in w.tags}
        
        # way'lerin zaman bilgisi
        ts = pd.Timestamp(w.timestamp)        
        visible = 1 if w.visible else 0
        highway = w.tags["highway"] if 'highway' in w.tags else None
        source = w.tags["source"] if 'source' in w.tags else None

        # dataframe için tablo başlıkları
        self.way_columns = ["type", "id", "version", "time", "user_id",
                            "highway", "source", "nodes", "visible"]
        self.ways.append(["way", w.id, w.version,  ts, w.uid,
                          highway, source, nodes, visible])

#osh_dosya = "deneme.osh.pbf"
osh_dosya = "ankara_highways.osh.pbf"
        
yol_tarih = HighwaysHandler()
yol_tarih.apply_file(osh_dosya, locations=True, idx="sparse_mem_array")

####  NODE, WAY VE RELATiON TABLOLARI ###############################################
# node bilgilerini içeren tablo
nodes_df = pd.DataFrame(yol_tarih.nodes, columns=yol_tarih.node_columns)
# way bilgilerini içeren tablo
ways_df = pd.DataFrame(yol_tarih.ways, columns=yol_tarih.way_columns)

nodes_df = nodes_df.sort_values(by=['type', 'id', 'time'])
ways_df = ways_df.sort_values(by=['type', 'id', 'time'])

In [3]:
ways_df.head()

,type,id,version,time,user_id,highway,source,nodes,visible
0,way,4412557,1,2007-05-18 21:08:02,6313,motorway,None,"[29279722, 26192695, 27002725, 27002726, 27002...",1
1,way,4412607,1,2007-05-21 21:08:21,6313,trunk,None,"[27003149, 27003150, 29279803, 29279802, 29279...",1
2,way,4412607,2,2007-12-15 22:37:02,12783,trunk,None,"[27003149, 27003150, 29279803, 29279802, 29279...",1
3,way,4412607,3,2008-11-10 22:06:24,11395,trunk,None,"[27003149, 27003150, 29279803, 29279802, 29279...",1
4,way,4412607,4,2009-02-13 22:19:35,18069,trunk,None,"[27003149, 27003150, 29279803, 29279802, 29279...",1


In [43]:
ways_df[ways_df["visible"]!=1]

,type,id,version,time,user_id,highway,source,nodes,visible


In [73]:
### FONKSİYON TANIMLAMALARI
def son_hali(df):
    """ Verinin en son halini verir. """
    data = df.groupby(['type','id'])['version'].max().reset_index()
    return pd.merge(data, df, on=['id','version'])

def iki_tarih_arasindaki_veri(df, bas, bit):
    """Dataframe içerisinde verilen iki tarih arasında olan elemanları verir.
    Eğer belirtilen iki tarih arasında değiştirilmiş, eklenmiş ya da silinmiş
    elemanlar elde edilmek isteniliyorsa kullanılması uygundur."""    
    data = df.query("time >= @bas & time <= @bit").reset_index()
    return data

def tarihli_son_hali(df, tarih):
    """Dataframe içerisinde, verilen tarih anında mevcut verilerin
    en son versiyonlu olanını verir.
    tarih: "2006-01-01" şeklinde string olarak ya da
    Timestamp('2006-09-01 00:00:00') nesnesi olarak girilebilir."""    
    data = (df.query("time <= @tarih")
                  .groupby(['type','id'])['version'].max().reset_index())
    return pd.merge(data, df, on=['type','id','version'])

### ay ve yila göre zaman ciftleri olusturmak
def zaman_ciftleri(bas, bit, frekans="yil", reverse=False):
    """ frekans="yil" ise; (freq="AS")
    {'2006': ('2006-01-01 00:00:00', '2007-01-01 00:00:00'),
     '2007': ('2007-01-01 00:00:00', '2008-01-01 00:00:00'), ........} seklinde
     
     frekans="ay" ise; (freq="1MS")
    {'2006-01': ('2006-01-01 00:00:00', '2006-02-01 00:00:00'), 
     '2006-02': ('2006-02-01 00:00:00', '2006-03-01 00:00:00'), ........} seklinde
     zaman ciflerini verir.     
     reverse: zamanları sondan başa doğru üretir.
    """
    sonuc = None
    if frekans=="yil":
        y = pd.date_range(bas, bit, freq="AS")
        z = {str(y.year[i]) : (y[i], y[i+1]) 
                          for i in range(len(y)) if i < len(y)-1 }
        sonuc = OrderedDict(sorted(z.items(), reverse=reverse))
    elif frekans=="ay":
        a = pd.date_range(bas, bit, freq="1MS")
        z = {str(a.year[i]) + "-" + str(a.month[i]).zfill(2) : (a[i], a[i+1]) 
                          for i in range(len(a)) if i < len(a)-1 }
        sonuc = OrderedDict(sorted(z.items(), reverse=reverse))
    elif frekans=="gun":
        g = pd.date_range(bas, bit, freq="D")
        z = {str(g.year[i]) + "-" + str(g.month[i]).zfill(2) + "-" + str(g.day[i]).zfill(2) :
             (g[i], g[i+1]) for i in range(len(g)) if i < len(g)-1 }
        sonuc = OrderedDict(sorted(z.items(), reverse=reverse))
    return sonuc


###
def node_sayisi(df, tarih, sadece_gorunen=True):
    sys.stdout.write(str(tarih.year - 1) + " ")
    data = tarihli_son_hali(df, tarih)
    nb_nodes = 0
    if sadece_gorunen: nb_nodes = len(data[data["visible"]==1])
    else: nb_nodes = len(data)        
    return nb_nodes

def node_sayisi_kronoloji(df, bas, bit, frekans, sadece_gorunen=True, reverse=False):
    aralik = zaman_ciftleri(bas, bit, frekans, reverse=reverse)
    st = [node_sayisi(df, aralik[i][1], sadece_gorunen) for i in aralik]
    column_name = "Gorunen Node" if sadece_gorunen else "Toplam Node"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st
#
def way_sayisi(df, tarih, sadece_gorunen=True):
    sys.stdout.write(str(tarih.year - 1) + " ")
    data = tarihli_son_hali(df, tarih)
    nb_way = 0
    if sadece_gorunen: nb_way = len(data[data["visible"]==1])
    else: nb_way = len(data)
    return nb_way

def way_sayisi_kronoloji(df, bas, bit, frekans="yil", sadece_gorunen=True):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [way_sayisi(df, aralik[i][1], sadece_gorunen) for i in aralik]
    column_name = "Gorunen Way" if sadece_gorunen else "Toplam Way"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st
#
def toplam_user_sayisi(df, tarih):
    """ Doğru netice için parametre olarak node listesi gönderilmelidir."""   
    sys.stdout.write(str(tarih.year - 1) + " ")
    data = tarihli_son_hali(df, tarih)
    nb_users = data["user_id"].nunique()
    return nb_users

def toplam_user_sayisi_kronoloji(df, bas, bit, frekans="yil"):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [toplam_user_sayisi(df, aralik[i][1]) for i in aralik]
    column_name = "User Sayisi"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st

def veri_giren_user_sayisi(df, bas, bit):
    data = iki_tarih_arasindaki_veri(df, bas, bit)
    nb_users = data["user_id"].nunique()
    return nb_users

def veri_giren_user_sayisi_kronoloji(df, bas, bit, frekans="yil"):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [veri_giren_user_sayisi(df, aralik[i][0], aralik[i][1]) for i in aralik]
    column_name = "User Sayisi"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st

def eklenen_node_sayisi(df, bas, bit):
    data = iki_tarih_arasindaki_veri(df, bas, bit)
    nb_nodes = len(data[data["version"]==1])    
    return nb_nodes

def eklenen_node_kronoloji(df, bas, bit, frekans):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [eklenen_node_sayisi(df, aralik[i][0], aralik[i][1]) for i in aralik]
    column_name = "Eklenen Node"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st
#
def eklenen_way_sayisi(df, bas, bit):
    data = iki_tarih_arasindaki_veri(df, bas, bit)
    nb_way = len(data[data["version"]==1])
    return nb_way

def eklenen_way_kronoloji(df, bas, bit, frekans):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [eklenen_way_sayisi(df, aralik[i][0], aralik[i][1]) for i in aralik]
    column_name = "Eklenen Way"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st
#
def degistirilen_node_sayisi(df, bas, bit):
    data = iki_tarih_arasindaki_veri(df, bas, bit)
    nb_nodes = len(data[data["version"]!=1])    
    return nb_nodes

def degistirilen_node_kronoloji(df, bas, bit, frekans):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [degistirilen_node_sayisi(df, aralik[i][0], aralik[i][1]) for i in aralik]
    column_name = "Degistirilen Node"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st
#
def degistirilen_way_sayisi(df, bas, bit):
    data = iki_tarih_arasindaki_veri(df, bas, bit)
    nb_way = len(data[data["version"]!=1])
    return nb_way

def degistirilen_way_kronoloji(df, bas, bit, frekans):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = [degistirilen_way_sayisi(df, aralik[i][0], aralik[i][1]) for i in aralik]
    column_name = "Degistirilen Way"    
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st

def kullanici_basina_degisiklik_kronoloji(df, bas, bit, frekans="yil", tur="Node"):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = []
    for i in aralik:
        data = iki_tarih_arasindaki_veri(df, aralik[i][0], aralik[i][1])
        user_sayisi = float(data["user_id"].nunique())
        oran = len(data[data["version"]!=1]) / user_sayisi if user_sayisi != 0 else None
        st.append(oran)
    
    column_name = "Kullanici Başına {} Degisiklik".format(tur)
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st

def kullanici_basina_eklenen_kronoloji(df, bas, bit, frekans="yil", tur="Node"):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = []
    for i in aralik:
        data = iki_tarih_arasindaki_veri(df, aralik[i][0], aralik[i][1])
        user_sayisi = float(data["user_id"].nunique())
        oran = len(data[data["version"]==1]) / user_sayisi if user_sayisi != 0 else None
        st.append(oran)
    
    column_name = "Kullanici Başına {} Eklenen".format(tur)
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st

def kullanici_basina_ekleme_degistirme_kronoloji(df, bas, bit, frekans="yil", tur="Node"):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = []
    for i in aralik:
        data = iki_tarih_arasindaki_veri(df, aralik[i][0], aralik[i][1])
        user_sayisi = float(data["user_id"].nunique())
        oran = len(data[data["version"]==1]) / user_sayisi if user_sayisi != 0 else None
        st.append(oran)
    
    column_name = "Kullanici Başına {} Eklenen".format(tur)
    st = pd.DataFrame(st, index=aralik.keys(), columns=[column_name])
    return st

def tur_kronoloji(df, bas, bit, frekans="yil", tur="highway"):
    aralik = zaman_ciftleri(bas, bit, frekans)
    st = pd.DataFrame()
    for i in aralik:
        data = tarihli_son_hali(df, aralik[i][1])
        c = data.groupby(tur).source.count()
        a = aralik[i][0]
        if frekans == "yil":            
            c.name = str(a.year)
        elif frekans == "ay":
            c.name = str(a.year) + "-" + str(a.month).zfill(2)
        else: # gun ise
            c.name = str(a.year) + "-" + str(a.month).zfill(2)+ "-" + str(a.day).zfill(2)
            
        st = st.append(c)
    return st

In [ ]:
bas = "2006-01-01"
bit = "2018-01-01"
frekans = "yil"
print("Görünen node sayısı belirleniyor...")
gorunen_node = node_sayisi_kronoloji(nodes_df, bas, bit, frekans, True, False)
print("\nToplam node sayısı belirleniyor...")
toplam_node = node_sayisi_kronoloji(nodes_df, bas, bit, frekans, False, False)

print("\nGörünen way sayısı belirleniyor...")
gorunen_way = way_sayisi_kronoloji(ways_df, bas, bit, frekans, True)
print("\nToplam way sayısı belirleniyor...")
toplam_way = way_sayisi_kronoloji(ways_df, bas, bit, frekans, False)

print("\nToplam user sayısı belirleniyor...")
toplam_users = toplam_user_sayisi_kronoloji(nodes_df, bas, bit, frekans)
print("\nVeri Giren user sayısı belirleniyor...")
users = veri_giren_user_sayisi_kronoloji(nodes_df, bas, bit, frekans)

print("\nEklenen node sayısı belirleniyor...")
eklenen_node = eklenen_node_kronoloji(nodes_df, bas, bit, frekans)
print("Eklenen way sayısı belirleniyor...")
eklenen_way = eklenen_way_kronoloji(ways_df, bas, bit, frekans)

print("Değiştirilen node sayısı belirleniyor...")
degistirilen_node = degistirilen_node_kronoloji(nodes_df, bas, bit, frekans)
print("Değiştirilen way sayısı belirleniyor...")
degistirilen_way = degistirilen_way_kronoloji(ways_df, bas, bit, frekans)

print("Kullanıcı başına node değişimleri hesaplanıyor...")
kullanici_basina_node_degisiklik = kullanici_basina_degisiklik_kronoloji(nodes_df, bas, bit, frekans, "Node")  
print("Kullanıcı başına way değişimleri hesaplanıyor...")
kullanici_basina_way_degisiklik = kullanici_basina_degisiklik_kronoloji(ways_df, bas, bit, frekans, "Way")

print("Kullanıcı başına node değişimleri hesaplanıyor...")
kullanici_basina_node_eklenen = kullanici_basina_eklenen_kronoloji(nodes_df, bas, bit, frekans, "Node")  
print("Kullanıcı başına way değişimleri hesaplanıyor...")
kullanici_basina_way_eklenen = kullanici_basina_eklenen_kronoloji(ways_df, bas, bit, frekans, "Way")

In [ ]:
print("Kullanıcı başına node değişimleri hesaplanıyor...")
kullanici_basina_node_ekleme_degistirme = kullanici_basina_ekleme_degistirme_kronoloji(nodes_df, bas, bit, frekans, "Node")  
print("Kullanıcı başına way değişimleri hesaplanıyor...")
kullanici_basina_way_ekleme_degistirme = kullanici_basina_ekleme_degistirme_kronoloji(ways_df, bas, bit, frekans, "Way")

In [37]:
print("Highway türleri belirleniyor...")
highway_turleri = tur_kronoloji(ways_df, bas, bit, frekans, tur="highway")

print("Highway için 'kaynak türleri' belirleniyor...")
kaynak_turleri = tur_kronoloji(ways_df, bas, bit, frekans, tur="source")

sayilar_tablosu = pd.DataFrame()
sayilar_tablosu = pd.concat([sayilar_tablosu, gorunen_node, toplam_node,
                             gorunen_way, toplam_way, eklenen_node,
                             eklenen_way, degistirilen_node, degistirilen_way,
                             toplam_users, users,
                             kullanici_basina_node_degisiklik,
                             kullanici_basina_way_degisiklik,
                             kullanici_basina_node_eklenen,
                             kullanici_basina_way_eklenen,
                             kullanici_basina_node_ekleme_degistirme,
                             kullanici_basina_way_ekleme_degistirme], axis=1)


Toplam user sayısı belirleniyor...
2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 
Eklenen node sayısı belirleniyor...


In [6]:
sayilar_tablosu.head()

,Gorunen Node,Toplam Node,Gorunen Way,Toplam Way,Eklenen Node,Eklenen Way,Degistirilen Node,Degistirilen Way,User Sayisi,Kullanici Başına Node Degisiklik,Kullanici Başına Way Degisiklik
2006,0,0,0,0,0,0,0,0,0,NaN,NaN
2007,0,0,0,0,0,0,0,0,0,NaN,NaN
2008,76,76,1,1,76,1,3,0,1,3.0,0.0
2009,719,960,2,4,884,3,242,13,1,242.0,13.0
2010,724,965,2,4,5,0,19,6,2,9.5,3.0


In [9]:
temp= 0
while os.path.exists("OSM İstatistik%s.xlsx".decode("utf-8") % temp): temp += 1
excel_file_name = "OSM İstatistik%s.xlsx".decode("utf-8") % temp
excel_writer = ExcelWriter(excel_file_name)
sayilar_tablosu.to_excel(excel_writer, sheet_name="Genel")
highway_turleri.to_excel(excel_writer, sheet_name="Highway Türleri")
kaynak_turleri.to_excel(excel_writer, sheet_name="Kaynak Türleri")
excel_writer.save()

print("İstatistik bilgiler '{}' dosyasına kaydedildi.".format(excel_file_name))